## **まず衛星データと地図表示しよう**

In [ ]:
import ee  # Import the Earth Engine library
import folium  # Import the Folium library
from google.colab import drive
import geopandas as gpd
import pandas as pd
import os, json
import numpy as np
import seaborn as sns


In [ ]:
drive.mount("/content/drive/")
input_dir = "GIS_Related_Study/NagoyaU_2025_RS/INPUT/"
input_path = "/content/drive/MyDrive/" + input_dir
output_dir = "GIS_Related_Study/NagoyaU_2025_RS/OUTPUT/"
output_path = "/content/drive/MyDrive/" + output_dir

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Earth Engineの認証と初期化（既に認証済みの場合は ee.Initialize() のみでOK）
# 吉田さんのGEEのカタログを流用
ee.Authenticate()
ee.Initialize(project='xxxx') #自身のPROJECT＿IDをいれてください（GMAILで認証がされます）

# --- メコン川流域の範囲 (チベットから南シナ海) ---
mekongBasin = ee.Geometry.Polygon([
    [90, 35], [115, 35], [115, 5], [90, 5]
])

# --- Landsat 5 (2000年) のデータ ---
landsat5 = (ee.ImageCollection('LANDSAT/LT05/C02/T1_TOA')
            .filterBounds(mekongBasin)
            .filterDate('2000-01-01', '2000-12-31')
            .median()
            .clip(mekongBasin))

# --- Landsat 8 (2020年) のデータ ---
landsat8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
            .filterBounds(mekongBasin)
            .filterDate('2020-01-01', '2020-12-31')
            .median()
            .clip(mekongBasin))

# --- NDWIの計算 ---
# Landsat 5の場合はバンド ['B3', 'B5']
ndwi2000 = landsat5.normalizedDifference(['B3', 'B5']).rename('NDWI_2000')
# Landsat 8の場合はバンド ['B3', 'B6']
ndwi2020 = landsat8.normalizedDifference(['B3', 'B6']).rename('NDWI_2020')

# --- NDWIのビジュアライズ設定 ---
ndwiParams = {'min': -0.5, 'max': 1, 'palette': ['white', 'blue']}

# --- Foliumで地図を作成 ---
m = folium.Map(location=[16, 104], zoom_start=5)

# Earth EngineのイメージをFoliumに追加するためのヘルパー関数
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Google Earth Engine',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# FoliumのMapクラスにメソッドを追加
folium.Map.add_ee_layer = add_ee_layer

# --- NDWIレイヤーの追加 ---
m.add_ee_layer(ndwi2000, ndwiParams, 'NDWI 2000')
m.add_ee_layer(ndwi2020, ndwiParams, 'NDWI 2020')

# レイヤーコントロールを追加してマップに重ね合わせたレイヤーを切り替え可能にする
folium.LayerControl().add_to(m)

# 作成した地図を表示
m